In [1]:
%load_ext autoreload
%autoreload 2

In [117]:
import os
from glob import glob
from subprocess import check_call
import textwrap

import pandas as pd

import oae_smyle
import project

In [134]:
df = pd.read_csv("problem-simulations.csv", index_col=("polygon", "basin", "start_date")).drop(columns=["index", "rmse"])

exclude = [
    (0, "North_Atlantic_basin", "0347-04"),
]
for x in exclude:
    df = df.drop(x)

df

,,,polygon_id
polygon,basin,start_date,
1,North_Atlantic_basin,0347-04,1
2,North_Atlantic_basin,0347-04,2
3,North_Atlantic_basin,0347-04,3
4,North_Atlantic_basin,0347-04,4
5,North_Atlantic_basin,0347-04,5
...,...,...,...
35,Southern_Ocean,0347-10,685
36,Southern_Ocean,0347-10,686
37,Southern_Ocean,0347-10,687


In [108]:
files = []
for row in df.iterrows():
    p = f"{row[0][0]:03d}"
    b = row[0][1]
    d = row[0][2].replace("0347", "1999")
    file = f"{project.dir_data}/alk-forcing/OAE-Efficiency-Map/alk-forcing-{b}.{p}-{d}.nc"
    assert os.path.exists(file), file
    files.append(file)
    
df["file"] = files
df

,,,polygon_id,file
polygon,basin,start_date,,
0,North_Atlantic_basin,0347-04,0,/global/cfs/projectdirs/m4746/Projects/OAE-Eff...
1,North_Atlantic_basin,0347-04,1,/global/cfs/projectdirs/m4746/Projects/OAE-Eff...
2,North_Atlantic_basin,0347-04,2,/global/cfs/projectdirs/m4746/Projects/OAE-Eff...
3,North_Atlantic_basin,0347-04,3,/global/cfs/projectdirs/m4746/Projects/OAE-Eff...
4,North_Atlantic_basin,0347-04,4,/global/cfs/projectdirs/m4746/Projects/OAE-Eff...
...,...,...,...,...
35,Southern_Ocean,0347-10,685,/global/cfs/projectdirs/m4746/Projects/OAE-Eff...
36,Southern_Ocean,0347-10,686,/global/cfs/projectdirs/m4746/Projects/OAE-Eff...
37,Southern_Ocean,0347-10,687,/global/cfs/projectdirs/m4746/Projects/OAE-Eff...


In [122]:
os.makedirs("case-info", exist_ok=True)
os.makedirs("case-info/build-out", exist_ok=True)


def submission_script(case, alk_forcing_file, refdate):
    header = textwrap.dedent(
        f"""\
    #!/bin/bash
    #SBATCH -J build.{case}
    #SBATCH -A {project.account}
    #SBATCH -e case-info/build-out/{case}-%J.out
    #SBATCH -o case-info/build-out/{case}-%J.out
    #SBATCH --time=01:00:00
    #SBATCH --mem=4GB
    #SBATCH --qos=shared
    #SBATCH --nodes=1
    #SBATCH --ntasks=1
    #SBATCH --constraint=cpu
    
    module load conda
    conda activate cworthy
    
    python oae_smyle.py --case {case} --alk-forcing-file {alk_forcing_file} --refdate {refdate}
    """
    )
    
    build_script = f"case-info/{case}.build"
    with open(build_script, "w") as fid:
        fid.write(header)
    return build_script


n = 0
build_scripts = []
for row in df.iterrows():
    polygon_ndx = f"{row[0][0]:03d}"
    basin = row[0][1]
    refdate = f"{row[0][2]}-01"
    alk_forcing_file = row[1]["file"]
    
    case = f"smyle.oae-map.{basin}.{polygon_ndx}.{refdate.replace('0347', '1999')}"

    build_script = submission_script(case, alk_forcing_file, refdate)
    check_call(["sbatch", build_script])
    
    build_scripts.append(build_scripts)
    break

build_scripts            

Submitted batch job 30041560
